In [2]:
from requests import post
from requests.exceptions import JSONDecodeError
import uuid
import os
from dotenv import load_dotenv

# Для ipynb явно загружаются переменные окружения
load_dotenv('.env')

GIGACHAT_AUTH_URL = os.environ.get("GIGACHAT_AUTH_URL")
GIGACHAT_AUTHORIZE_KEY = os.environ.get("GIGACHAT_AUTHORIZE_KEY")
GIGACHAT_REQUEST_URL = os.environ.get("GIGACHAT_REQUEST_URL")
GIGACHAT_MODEL = os.environ.get("GIGACHAT_MODEL")

if not GIGACHAT_REQUEST_URL:
    raise ValueError("GIGACHAT_REQUEST_URL не установлена в окружении")

if not GIGACHAT_MODEL:
    raise ValueError("GIGACHAT_MODEL не установлена в окружении")

if not GIGACHAT_AUTH_URL:
    raise ValueError("GIGACHAT_AUTH_URL не установлен в окружении")

if not GIGACHAT_AUTHORIZE_KEY:
    raise ValueError("GIGACHAT_AUTHORIZE_KEY не установлен в окружении")

# Обновлять токен доступа, если до окончания действия осталось 60 секунд
REFRESH_ACCESS_TOKEN_BEFORE_S = 60
__access_token = ""
__access_token_expires = ""

def refresh_access_token():
    """Refresh access_token for llm and set access_token_expires time

    Returns:
        Boolean: 
            True if refresh is success
            False if token is not refreshed
    """

    global __access_token
    global __access_token_expires

    headers = {
        "Content-Type": "application/x-www-form-urlencoded",
        "Accept": "application/json",
        "RqUID": f"{uuid.uuid4()}",
        "Authorization": f"Basic {GIGACHAT_AUTHORIZE_KEY}"
    }

    payload = {
        "scope": "GIGACHAT_API_PERS"
    }

    try:
        response = post(
            url=GIGACHAT_AUTH_URL,
            headers=headers,
            data=payload
        )

        response_json = response.json()

        __access_token = response_json.get("access_token")
        __access_token_expires = response_json.get("expires_at")

        if __access_token is None or __access_token_expires is None:
            print("Ошибка при выполнении запроса на обновление токена доступа",
                  "\nВ полученных данных отсутствуют 'access_token' или 'expires_at'",
                  f"\nОтвет сервиса: {response_json}")
            return False
    except JSONDecodeError as ex:
        print(f"Ошибка при парсинге ответа от сервиса при выполнении запроса на обновление токена доступа\n{ex}")
        return False
    except Exception as ex:
        print(f"Ошибка при выполнении запроса на обновление токена доступа\n{ex}")
        return False
    
    return True


In [3]:
is_refresh_success = refresh_access_token()
print(is_refresh_success)

True


In [4]:
from datetime import datetime

# При обращении к функции передачи сообщения нужна проверка действительности токена и его обновление

def refresh_access_token_if_expire(func):
    def wrapper(*args, **kwargs):
        # Проверка действительности токена
        # Преобразование в секунды для корректной работы с datetime
        token_expire_time_s = __access_token_expires / 1000.0
        token_expire_datetime = datetime.fromtimestamp(token_expire_time_s)
        # Остаток времени до истечения токена в секундах
        time_left_s = (token_expire_datetime - datetime.now()).total_seconds()

        print(
                "Refreshing access token",
                f"Expire datetime is: {token_expire_datetime}",
                f"Seconds left: {time_left_s}"
            )
        
        # Если действие токена скоро закончится или уже закончилось - обновить
        if time_left_s < REFRESH_ACCESS_TOKEN_BEFORE_S:
            print(
                "Refreshing access token",
                f"Expire datetime is: {token_expire_datetime}",
                f"Seconds left: {time_left_s}"
            )
            refresh_access_token()
        
        return func(*args, **kwargs)

    return wrapper

In [5]:
# Нужна выгрузка истории сообщений по достижении определенного лимита, чтобы не хранить всё в оперативной памяти
# Нужно лимитировать историю сообщений, передаваемую LLM

# Формат хранения истории сообщений
# message_history = [
#     {
#         "role": "user",
#         "content": "request text"
#     }
# ]

import json

message_history = []

@refresh_access_token_if_expire
def bot_make_request(request_message: str):
    message_history.append({
        "role": "user",
        "content": request_message
    })

    headers = {
        "Content-Type": "application/json",
        "Accept": "application/json",
        "Authorization": f"Bearer {__access_token}"
    }

    body = json.dumps({
        "model": GIGACHAT_MODEL,
        "messages": message_history[-10:] # Последние 10 сообщений из истории
    })

    try:
        response = post(
            url=GIGACHAT_REQUEST_URL,
            headers=headers,
            data=body
        )

        response_json = response.json()

        assistant_message = response_json.get("choices")[0].get("message").get("content")

        if assistant_message is None:
            print(f"[ERROR] В полученных данных отсутствует сообщение модели")
            return False
        
        # Добавить ответ модели в историю сообщений
        message_history.append({
            "role": "assistant",
            "content": assistant_message
        })
    except JSONDecodeError as ex:
        print(f"[ERROR] Ошибка при парсинге ответа от сервиса при выполнении запроса на обновление токена доступа\n{ex}")
        return False
    except Exception as ex:
        print("[ERROR] Ошибка при выполнении запроса к LLM", ex)
        return False
    
    return True

In [6]:
bot_make_request("Привет! Какие у тебя возможности?")

Refreshing access token Expire datetime is: 2026-02-16 21:10:50.624000 Seconds left: 1799.553021


True

In [7]:
print(message_history)

[{'role': 'user', 'content': 'Привет! Какие у тебя возможности?'}, {'role': 'assistant', 'content': 'Привет!\n\nВот основные мои возможности и форматы работы:\n\n### 📌 Текст\n\n- Работаю с русским языком: отвечаю на запросы, пишу тексты разных стилей (от рассказов до официальных документов).\n- Решаю задачи разной сложности — от простых вопросов до сложных вычислений, программирования и логики.\n- Могу объяснить материал подробно, пошагово разбирая решение.\n\n### 🎨 Генерация изображений\n\nУ меня есть возможность создавать уникальные изображения на основе твоих запросов. Например, нарисовать персонажа, пейзаж или иллюстрацию по твоему описанию.\n\n### 💻 Программирование\n\nМогу написать код на популярных языках программирования (Python, JavaScript, C++, PHP, SQL и другие), исправить ошибки, оптимизировать программы и объяснить, как работает конкретный участок кода.\n\n---\n\nЕсть конкретная задача или вопрос? Пиши!'}]


In [8]:
bot_make_request("Составь план действий. Хочу написать скрипт, который будет отправлять в группу в Telegram информацию о загруженности ВМ (cpu, ram, дисковое пространство). Хочу использовать python-telegram-bot")

Refreshing access token Expire datetime is: 2026-02-16 21:10:50.624000 Seconds left: 1797.322725


True

In [9]:
from pathlib import Path

__username = None
__save_message_history = False

def bot_load_user_config(config_path="conf/user_settings.json"):
    global __username
    global __save_message_history

    path = Path(config_path)

    if (path.exists()):
        try:
            with open(path, "r", encoding="utf-8") as settings_file:
                settings = json.load(settings_file)

                __username = settings.get("username")
                __save_message_history = settings.get("save_message_history", False)

                if __username is None:
                    print(f"[ERROR] Не удается найти имя пользователя в файле конфигурации {config_path} (username)")
                    return False
                
                if __save_message_history is None:
                    print(f"[WARN] Флаг сохранения истории сообщений установлен в False, так как не был найден в файле конфигурации {config_path} (save_message_history)")
                
                return True
        except JSONDecodeError as ex:
            print("[ERROR] Ошибка при парсинге файла конфигурации в JSON", ex)
        except Exception as ex:
            print("[ERROR] Непредвиденная ошибка при чтении файла конфигурации", ex)
    else:
        print(f"[ERROR] Не удается найти файл конфигурации пользователя по пути {path}")
        return False

In [10]:
bot_load_user_config()

True

In [11]:
print(__username, __save_message_history)

host True


In [12]:
from enum import Enum

class FileFormat(Enum):
    JSON = "json"
    TEXT = "txt"

# Функция для сохранения файла
def utils_save_file(path: Path, format: str, data) -> bool:
    print(path)
    if not path.exists():
        print("[WARN] Путь к файлу не существует. Попытка создать файл")
        try:
            path.touch()
        except Exception as ex:
            print("[ERROR] Ошибка при создании файла", ex)
            return False
    

    if format == FileFormat.JSON:
        try:
            with path.open("w", encoding="utf-8") as file:
                json.dump(data, file, indent=4, ensure_ascii=False)
        except Exception as ex:
            print("[ERROR] Ошибка при сохранении JSON файла")
            return False
    elif format == FileFormat.TEXT:
        try:
            with path.open("w", encoding="utf-8") as file:
                file.write(str(data))
        except Exception as ex:
            print("[ERROR] Ошибка при сохранении текстового файла")
            return False
    else:
        print("[ERROR] Неизвестный формат файла. Сохранение не будет выполнено")
    
    return True

# Функция для сохранения истории сообщений
def bot_save_message_history(message_history: list = None) -> bool:
    try:
        if message_history is None:
            print("[ERROR] Не задана история сообщений для сохранения")
            return False
        elif len(message_history) == 0:
            print("[WARN] История сообщений пуста. Сохранение не будет выполнено")
            return False
        elif __username is None:
            print("[ERROR] Пользователь для сохранения истории сообщений не известен")
            return False
    except Exception as ex:
        print("[ERROR] Ошибка при сохранении истории сообщений", ex)

    path = Path(f"data/history/{__username}.json")
    utils_save_file(path, FileFormat.JSON, message_history)

    return True

In [13]:
import copy

@refresh_access_token_if_expire
def bot_make_system_request(request_message: str, message_context: list = None) -> dict:
    """Make request to llm without user message history editing

    Args:
        request_message (str): request to llm
        message_history (list, optional): previous messages to|from llm. Defaults to None. If need to use the messages for context

    Returns:
        dict: llm answer in format: {
            "role": "assistant",
            "message": assistant_message
        }

        None if something went wrong
    """
    if message_context is not None:
        request_messages = copy.deepcopy(message_context)
    else:
        request_messages = []

    # Т.к. системное сообщение может быть направлено только первым, запросы программы (не пользователя) также передаются под ролью user
    request_messages.append({
        "role": "user",
        "content": request_message
    })

    print(request_messages)

    headers = {
        "Content-Type": "application/json",
        "Accept": "application/json",
        "Authorization": f"Bearer {__access_token}"
    }

    body = json.dumps({
        "model": GIGACHAT_MODEL,
        "messages": request_messages
    })

    try:
        response = post(
            url=GIGACHAT_REQUEST_URL,
            headers=headers,
            data=body
        )

        response_json = response.json()

        assistant_message = response_json.get("choices")[0].get("message").get("content")

        if assistant_message is None:
            print(f"[ERROR] В полученных данных отсутствует сообщение модели")
            return None
        
        # Вернуть ответ модели
        return {
            "role": "assistant",
            "message": assistant_message
        }
    except JSONDecodeError as ex:
        print(f"[ERROR] Ошибка при парсинге ответа от сервиса при выполнении запроса на обновление токена доступа\n{ex}")
        return None
    except Exception as ex:
        print("[ERROR] Ошибка при выполнении запроса к LLM", ex)
        return None

In [14]:
# TODO

__context = None
__context_prompt = None

# Функция для генерации контекста на основе предыдущих сообщений
def bot_generate_context(message_history: list = None):
    global __context

    if __context_prompt is None:
        print("[ERROR] Промпт для генерации контекста не задан")
        return False

    response_message = bot_make_system_request(__context_prompt, message_context=message_history)

    if response_message is not None:
        __context = response_message.get("message")
    else:
        return False

    return True



# Функция для сохранения контекста (под капотом использовать функцию сохранения файла)
def bot_save_context(context_data: str):
    path = Path(f"data/context/{__username}")

    if context_data is not None:
        utils_save_file(path=path, format=FileFormat.TEXT, data=context_data)
    else:
        print("[WARN] Контекст пуст, сохранение не будет произведено")

def bot_load_context_prompt(context_path="conf/app/context_prompt"):
    global __context_prompt

    path = Path(context_path)

    if path.exists():
        with path.open("r", encoding="utf-8") as file:
            data = file.read()

            if data:
                __context_prompt = data
                return True
            else:
                print("[ERROR] Файл промпта для генерации контекста пуст. Промпт генерации контекста не задан")
                return False
    else:
        print(f"[ERROR] Указанный путь к файлу контекста не существует ({path})")
        return False

In [15]:
bot_load_context_prompt()

True

In [16]:
print(__context_prompt)

Сгенерируй контекст на основе истории сообщений
Контекст должен включать сведения о пользователе, если они присутствуют в истории сообщений
Также в контексте необходимо зафиксировать предмет разговора - о чем ведется речь в истории сообщений
Если в истории сообщений есть информация о конкретных предметах, технологиях и т.д., необходимо зафиксировать это в контексте

Пример контекста:
Пользователь Василий интересуется разработкой на Python. Он задает вопросы о том как сохранять файлы в Python, выполнять запросы к API.
Также Василий планирует применить полученные знания в рабочих задачах. Он хочет создать скрипт на Python, который будет присылать уведомления в Telegram
о нехватке ресурсов на хосте на базе ОС Linux. Можно подсказывать ему о технологиях, которые позволят ему реализовать свою идею.

Сделай контекст как можно меньше, с сохранением смысла

Не используй это сообщение в качестве источника информации при генерации контекста.
Не включай в контекст информацию в которой не уверен.


In [17]:
bot_generate_context(message_history=message_history)

Refreshing access token Expire datetime is: 2026-02-16 21:10:50.624000 Seconds left: 1789.297498
[{'role': 'user', 'content': 'Привет! Какие у тебя возможности?'}, {'role': 'assistant', 'content': 'Привет!\n\nВот основные мои возможности и форматы работы:\n\n### 📌 Текст\n\n- Работаю с русским языком: отвечаю на запросы, пишу тексты разных стилей (от рассказов до официальных документов).\n- Решаю задачи разной сложности — от простых вопросов до сложных вычислений, программирования и логики.\n- Могу объяснить материал подробно, пошагово разбирая решение.\n\n### 🎨 Генерация изображений\n\nУ меня есть возможность создавать уникальные изображения на основе твоих запросов. Например, нарисовать персонажа, пейзаж или иллюстрацию по твоему описанию.\n\n### 💻 Программирование\n\nМогу написать код на популярных языках программирования (Python, JavaScript, C++, PHP, SQL и другие), исправить ошибки, оптимизировать программы и объяснить, как работает конкретный участок кода.\n\n---\n\nЕсть конкретна

True

In [18]:
print(__context)

Иван обсуждает с коллегами способы автоматизации мониторинга серверов Linux и отправки уведомлений о высокой нагрузке через Telegram. Иван ранее задавал вопросы о работе с библиотекой `python-telegram-bot` и сборе метрик загрузки CPU, RAM и дискового пространства. Участники беседы планируют интегрировать мониторинг нагрузки на серверы в рабочие процессы компании.


In [19]:
bot_save_context(__context)

data\context\host


In [21]:
bot_save_message_history(message_history)

data\history\host.json
[WARN] Путь к файлу не существует. Попытка создать файл


True